<a href="https://colab.research.google.com/github/samleitermann/DataScience/blob/main/Spotify_Wrapped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# Install the datascience library
!pip install datascience
# Import necessary libraries
from datascience import *
import numpy as np
from google.colab import userdata

# Set up matplotlib for inline plotting and apply a style
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [6]:
# Read the listening data from a CSV file hosted on GitHub
listening_data = Table.read_table('https://raw.githubusercontent.com/samleitermann/DataScience/refs/heads/main/SLLScrobbles.csv')

In [7]:
# Display the first 10 rows of the listening data
listening_data.show(10)

Date,Time,Artist,Album,Track
6-Oct-25,02:20,Sofia Isella,Everybody Supports Women,Everybody Supports Women
5-Oct-25,20:17,Imagine Dragons,Mercury - Acts 1 & 2,Bones
4-Oct-25,23:24,Mariah Carey,Merry Christmas,All I Want for Christmas Is You
4-Oct-25,20:59,Rainbow Strings,Sky Color,Sky Color
4-Oct-25,19:22,Rainbow Strings,My Cloud,My Cloud
4-Oct-25,19:19,Eternal Warriors,Checkpoint,A Pause in the Journey
4-Oct-25,19:16,Ruttik Dhakate,Beneath the Quiet Stars,Beneath the Quiet Stars
4-Oct-25,19:11,Moving Gradients,Dream Glow,Dream Glow
4-Oct-25,19:09,Muse K,Ambient Dreamscape,Blissful Lotus Blossoms
4-Oct-25,19:07,Casen Wilham,Relaxing Music,Relaxing Music to Work To


In [9]:
# Group the listening data by 'Artist' and 'Track' to get unique tracks and their counts
unique_tracks = listening_data.group(['Artist','Track'])

In [10]:
# Display the first 10 rows of the unique_tracks table
unique_tracks.show(10)

Artist,Track,count
"""Les Misérables Original London Cast"" Ensemble",One Day More,1
"""Les Misérables Original London Cast"" Male Ensemble",Prologue: Work Song,1
*NSYNC,Bye Bye Bye,3
*NSYNC,"Merry Christmas, Happy Holidays",1
.38 Special,Back Where You Belong - Edited Version,1
.38 Special,If I'd Been the One,2
11 Acorn Lane,You Make Me Happy (Electro Swing Remix),1
2Pac,Hail Mary,1
2Pac,Thugz Mansion - 2Pac Original/Acoustic,5
4 Non Blondes,What's Up?,1


In [11]:
# Install or upgrade the spotipy library
pip install spotipy --upgrade

In [13]:
# Import the spotipy library and its authentication module
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [61]:
cid = userdata.get("Spotifycid")
secret = userdata.get("SpotifySecret")
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [17]:
# Import tqdm for progress bar
from tqdm.notebook import tqdm

# Initialize arrays to store song lengths and popularity
song_lengths = np.zeros(unique_tracks.num_rows)
song_pops = np.zeros(unique_tracks.num_rows)
# Iterate through each unique track with a progress bar
for row_index in tqdm(np.arange(0, unique_tracks.num_rows)):
  # Get song and artist names
  song_name = unique_tracks.row(row_index).item('Track')
  artist_name = unique_tracks.row(row_index).item('Artist')
  # Search Spotify for the track
  track_id = sp.search(q='artist:' + artist_name + ' track:' + song_name, type='track')
  # If track is found, extract length and popularity
  if len(track_id['tracks']['items']) > 0:
    song_info = track_id['tracks']['items'][0]
    song_lengths[row_index] = song_info['duration_ms']
    song_pops[row_index] = song_info['popularity']
  # If track is not found, set length and popularity to -1
  else:
    song_lengths[row_index] = -1
    song_pops[row_index] = -1

  0%|          | 0/6336 [00:00<?, ?it/s]

In [19]:
# Add 'Length' and 'Popularity' columns to the unique_tracks table
unique_tracks = unique_tracks.with_columns('Length', song_lengths, 'Popularity', song_pops)

# Display the first 10 rows of the updated unique_tracks table
unique_tracks.show(10)

Artist,Track,count,Length,Popularity
"""Les Misérables Original London Cast"" Ensemble",One Day More,1,216133,52
"""Les Misérables Original London Cast"" Male Ensemble",Prologue: Work Song,1,203240,36
*NSYNC,Bye Bye Bye,3,200560,71
*NSYNC,"Merry Christmas, Happy Holidays",1,255306,45
.38 Special,Back Where You Belong - Edited Version,1,242160,47
.38 Special,If I'd Been the One,2,235640,56
11 Acorn Lane,You Make Me Happy (Electro Swing Remix),1,179010,21
2Pac,Hail Mary,1,312373,66
2Pac,Thugz Mansion - 2Pac Original/Acoustic,5,252293,47
4 Non Blondes,What's Up?,1,295533,82


In [21]:
# Define a function to combine two strings
def combine_str(str1, str2):
  return str1+str2

# Create a unique identifier column for unique_tracks by combining 'Artist' and 'Track'
unique_tracks = unique_tracks.with_column('unique', unique_tracks.apply(combine_str,'Artist','Track'))
# Create a unique identifier column for listening_data by combining 'Artist' and 'Track'
listening_data = listening_data.with_column('unique', listening_data.apply(combine_str,'Artist','Track'))

In [22]:
# Display the first 3 rows of unique_tracks
unique_tracks.show(3)
# Sort listening_data by 'unique' and display the first 3 rows
listening_data.sort('unique').show(3)

Artist,Track,count,Length,Popularity,unique
"""Les Misérables Original London Cast"" Ensemble",One Day More,1,216133,52,"""Les Misérables Original London Cast"" EnsembleOne Day More"
"""Les Misérables Original London Cast"" Male Ensemble",Prologue: Work Song,1,203240,36,"""Les Misérables Original London Cast"" Male EnsembleProlo ..."
*NSYNC,Bye Bye Bye,3,200560,71,*NSYNCBye Bye Bye


Date,Time,Artist,Album,Track,unique
28-Mar-25,23:20,"""Les Misérables Original London Cast"" Ensemble",Les Misérables (Original 1985 London Cast Recording),One Day More,"""Les Misérables Original London Cast"" EnsembleOne Day More"
17-Feb-25,20:13,"""Les Misérables Original London Cast"" Male Ensemble",Les Misérables (Original 1985 London Cast Recording),Prologue: Work Song,"""Les Misérables Original London Cast"" Male EnsembleProlo ..."
4-May-19,16:14,*NSYNC,No Strings Attached,Bye Bye Bye,*NSYNCBye Bye Bye


In [23]:
# Join listening_data with unique_tracks on the 'unique' column, dropping redundant columns from unique_tracks
listening_data_final = listening_data.join('unique', unique_tracks.drop('Artist','Track','count'))

In [24]:
# Display the first 3 rows of the final combined data
listening_data_final.show(3)

unique,Date,Time,Artist,Album,Track,Length,Popularity
"""Les Misérables Original London Cast"" EnsembleOne Day More",28-Mar-25,23:20,"""Les Misérables Original London Cast"" Ensemble",Les Misérables (Original 1985 London Cast Recording),One Day More,216133,52
"""Les Misérables Original London Cast"" Male EnsembleProlo ...",17-Feb-25,20:13,"""Les Misérables Original London Cast"" Male Ensemble",Les Misérables (Original 1985 London Cast Recording),Prologue: Work Song,203240,36
*NSYNCBye Bye Bye,4-May-19,16:14,*NSYNC,No Strings Attached,Bye Bye Bye,200560,71


In [25]:
# Count the number of rows in the final data where 'Length' is not -1 (meaning track info was found)
listening_data_final.where('Length', are.not_equal_to(-1)).num_rows

10597

In [26]:
# Group the listening data by 'Artist' to get unique artists and their counts
unique_artists = listening_data.group('Artist')
# Display the first 3 rows of the unique_artists table
unique_artists.show(3)

Artist,count
"""Les Misérables Original London Cast"" Ensemble",1
"""Les Misérables Original London Cast"" Male Ensemble",1
*NSYNC,4


In [28]:
# Import tqdm for progress bar
from tqdm.notebook import tqdm

# Initialize a list to store artist genres
artist_genre = ["" for x in range(unique_artists.num_rows)]
# Iterate through each unique artist with a progress bar
for row_index in tqdm(np.arange(0, unique_artists.num_rows)):
  # Get the artist name
  artist_name = unique_artists.row(row_index).item('Artist')
  # Search Spotify for the artist
  artist_id = sp.search(q='artist:' + artist_name, type='artist')
  # If artist is found, extract their genres
  if len(artist_id['artists']['items']) > 0:
    artist_genre[row_index] = artist_id['artists']['items'][0]['genres']
  # If artist is not found, assign an empty list for genres
  else:
    artist_genre[row_index] = []

  0%|          | 0/2608 [00:00<?, ?it/s]

In [29]:
# Add the 'Genres' column to the unique_artists table
unique_artists = unique_artists.with_column('Genres', artist_genre)
# Display rows 1000, 1001, and 1002 of the unique_artists table
unique_artists.take(1000,1001,1002).show(3)

Artist,count,Genres
Jackie Wilson,9,"['northern soul', 'motown', 'classic soul', 'soul', 'doo ..."
Jackson Browne,2,"['yacht rock', 'folk rock', 'soft rock', 'singer-songwri ..."
Jacky Terrasson,1,"['jazz', 'french jazz']"


In [30]:
# Join the final listening data with unique artists on the 'Artist' column
listening_data_final = listening_data_final.join('Artist', unique_artists)

In [31]:
# Display the first 3 rows of the updated final listening data
listening_data_final.show(3)

Artist,unique,Date,Time,Album,Track,Length,Popularity,count,Genres
"""Les Misérables Original London Cast"" Ensemble","""Les Misérables Original London Cast"" EnsembleOne Day More",28-Mar-25,23:20,Les Misérables (Original 1985 London Cast Recording),One Day More,216133,52,1,['musicals']
"""Les Misérables Original London Cast"" Male Ensemble","""Les Misérables Original London Cast"" Male EnsembleProlo ...",17-Feb-25,20:13,Les Misérables (Original 1985 London Cast Recording),Prologue: Work Song,203240,36,1,['musicals']
*NSYNC,*NSYNCBye Bye Bye,4-May-19,16:14,No Strings Attached,Bye Bye Bye,200560,71,4,[]


In [32]:
# Initialize a list to store all genres from all artists
all_genres = []
# Iterate through the 'Genres' column of unique_artists and extend the all_genres list
for genre_list in unique_artists.column('Genres'):
  all_genres = all_genres + genre_list

In [33]:
# Create a temporary table with all the extracted genres
temp_table = Table().with_column('Genres', all_genres)

# Group the temporary table by 'Genres' to count the occurrences of each genre
temp_table = temp_table.group('Genres')

In [34]:
# Sort the temporary table by genre count in descending order and display the top 3
temp_table.sort('count', descending = True).show(3)

Genres,count
jazz,205
christmas,103
children's music,81


In [44]:
# Define a function to determine the primary genre based on genre counts
def primary_genre(genre_list):
  if len(genre_list) == 0:
    return ""
  max_count = -1
  max_genre = ""
  # Iterate through the genres of an artist
  for genre in genre_list:
    # Find the count of the current genre in the overall genre counts table
    count = temp_table.where('Genres', are.equal_to(genre)).column('count').item(0)
    # Update max_count and max_genre if the current genre has a higher count
    if count > max_count:
      max_count = count
      max_genre = genre
  return max_genre

# Apply the primary_genre function to the 'Genres' column of unique_artists
primary_genres = unique_artists.apply(primary_genre, 'Genres')

array(['musicals', 'musicals', '', ..., '', "children's music", ''],
      dtype='<U21')

In [52]:
# Add the 'Primary Genre' column to the unique_artists table
unique_artists = unique_artists.with_column('Primary Genre', primary_genres)
unique_artists.show(3)

Artist,count,Genres,Primary Genre
"""Les Misérables Original London Cast"" Ensemble",1,['musicals'],musicals
"""Les Misérables Original London Cast"" Male Ensemble",1,['musicals'],musicals
*NSYNC,4,[],


In [53]:
# Join the final listening data with unique artists on the 'Artist' column to add 'Primary Genre'
listening_data_final = listening_data_final.join('Artist', unique_artists.select('Artist', 'Primary Genre'))

# Display the first few rows of the updated final listening data
listening_data_final.show(3)

Artist,Date,Time,Album,Track,Length,Popularity,Genres,Primary Genre
"""Les Misérables Original London Cast"" Ensemble",28-Mar-25,23:20,Les Misérables (Original 1985 London Cast Recording),One Day More,216133,52,['musicals'],musicals
"""Les Misérables Original London Cast"" Male Ensemble",17-Feb-25,20:13,Les Misérables (Original 1985 London Cast Recording),Prologue: Work Song,203240,36,['musicals'],musicals
*NSYNC,4-May-19,16:14,No Strings Attached,Bye Bye Bye,200560,71,[],


In [54]:
# Drop redundant columns from the final listening data table
listening_data_final = listening_data_final.drop('count','unique','Genres_2','Track _2')


# Display the first 3 rows of the cleaned final listening data
listening_data_final.show(3)

Artist,Date,Time,Album,Track,Length,Popularity,Genres,Primary Genre
"""Les Misérables Original London Cast"" Ensemble",28-Mar-25,23:20,Les Misérables (Original 1985 London Cast Recording),One Day More,216133,52,['musicals'],musicals
"""Les Misérables Original London Cast"" Male Ensemble",17-Feb-25,20:13,Les Misérables (Original 1985 London Cast Recording),Prologue: Work Song,203240,36,['musicals'],musicals
*NSYNC,4-May-19,16:14,No Strings Attached,Bye Bye Bye,200560,71,[],


In [55]:
# Save the final listening data table to a CSV file
listening_data_final.to_csv('all_streams_data.csv')

In [ ]:
old_data = Table.read_table('https://raw.githubusercontent.com/anewlon-deerfield/Data-Science-Materials/refs/heads/main/all_streams_data.csv')
old_data.show(3)

Month,Date,Year,Time,Artist,Album,Track,Length,Popularity,Genres,Primary Genre
Jul,31,2018,1:13,!llmind,The Hamilton Mixtape,Take A Break - Interlude,48386,38,[],nan
Aug,26,2017,14:44,'13: The Musical - Original West End Cast Recording' Company,13: The Musical (Original West End Cast Recording),Getting Ready,-1,-1,[],nan
May,10,2017,16:21,'13: The Musical - Original West End Cast Recording' Company,13: The Musical (Original West End Cast Recording),Getting Ready,-1,-1,[],nan


In [ ]:
# Define a function to combine date components into a single string
def combine_date(day,month,year):
  return str(day)+'-'+month+'-'+(str(year)[2:])

# Apply the combine_date function to create a 'Date' column in old_data and drop the original date components
old_data = old_data.with_column('Date',old_data.apply(combine_date,'Date','Month','Year')).drop('Month','Year')
# Display the first 3 rows of the updated old_data
old_data.show(3)
# Display the first 3 rows of the final listening data (for comparison)
listening_data_final.show(3)

ValueError: The column "Month" is not in the table. The table contains these columns: Date, Time, Artist, Album, Track, Length, Popularity, Genres, Primary Genre

In [ ]:
# Display the first 3 rows of old_data
old_data.show(3)
# Display the first 3 rows of listening_data_final
listening_data_final.show(3)
# Relabel the 'Track ' column to 'Track' in listening_data_final (fixing a potential typo)
listening_data_final = listening_data_final.relabeled('Track ','Track')

Date,Time,Artist,Album,Track,Length,Popularity,Genres,Primary Genre
31-Jul-18,1:13,!llmind,The Hamilton Mixtape,Take A Break - Interlude,48386,38,[],nan
26-Aug-17,14:44,'13: The Musical - Original West End Cast Recording' Company,13: The Musical (Original West End Cast Recording),Getting Ready,-1,-1,[],nan
10-May-17,16:21,'13: The Musical - Original West End Cast Recording' Company,13: The Musical (Original West End Cast Recording),Getting Ready,-1,-1,[],nan


Artist,Date,Time,Album,Track,Length,Popularity,Genres,Primary Genre
#1 Dads,7-May-24,0:19,About Face,Camberwell,229960,41,['australian indie'],australian indie
.Wave.,17-Sep-24,3:06,Pretender,Engadine Valley,195698,10,['florida rap'],florida rap
.anxious.,6-Oct-24,16:11,3am EP,memories,116756,34,['lo-fi beats'],lo-fi beats


In [ ]:
# Append the final listening data to the old data
listening_data_final = old_data.append(listening_data_final)

In [58]:
# Read the combined data from a CSV file hosted on GitHub
data = Table.read_table('https://raw.githubusercontent.com/anewlon-deerfield/Data-Science-Materials/refs/heads/main/all_streams_data.csv')

In [ ]:
# Display the first 5 rows of the combined data
data.show(5)

Date,Time,Artist,Album,Track,Length,Popularity,Genres,Primary Genre
31-Jul-18,1:13,!llmind,The Hamilton Mixtape,Take A Break - Interlude,48386,38,[],nan
26-Aug-17,14:44,'13: The Musical - Original West End Cast Recording' Company,13: The Musical (Original West End Cast Recording),Getting Ready,-1,-1,[],nan
10-May-17,16:21,'13: The Musical - Original West End Cast Recording' Company,13: The Musical (Original West End Cast Recording),Getting Ready,-1,-1,[],nan
22-Oct-17,17:46,'In The Heights' Original Broadway Company,In The Heights (Original Broadway Cast Recording),Breathe,244066,52,['broadway'],broadway
2-Sep-19,20:38,'Legally Blonde' Ensemble,Legally Blonde The Musical (Original Broadway Cast Recor ...,Blood In The Water,289960,45,['broadway'],broadway


In [ ]:
# Group the data by 'Artist' and 'Track', sort by count in descending order, and show the top 6
data.group(['Artist','Track']).sort('count',descending = True).show(6)

Artist,Track,count
nan,nan,2054
Modern Baseball,Your Graduation,258
Heyrocco,Melt,176
Modern Baseball,Mass,172
Modern Baseball,"Fine, Great",171
Pinegrove,Old Friends,168


In [ ]:
# Group the data by 'Artist', sort by count in descending order, and show the top 6
data.group('Artist').sort('count',descending = True).show(6)

Artist,count
Modern Baseball,3421
The Front Bottoms,2617
nan,2054
Heyrocco,1478
Sorority Noise,1121
The Frights,1084


In [ ]:
# Group the data by 'Primary Genre', sort by count in descending order, and show the top 6
data.group('Primary Genre').sort('count',descending = True).show(6)

Primary Genre,count
pov: indie,16298
nan,11813
pop,9253
australian indie,3791
pop punk,3147
bubblegrunge,3141


In [ ]:
# Get the number of rows in the data
data.num_rows

100820

In [ ]:
# Calculate the total length of tracks with length > 0 in days
sum(data.where('Length',are.above(0)).column('Length'))/1000/60/60/24

240.96876591435182

In [ ]:
# Calculate the total length of all tracks in days
sum(data.column('Length'))/1000/60/60/24

346994.96706666669

In [ ]:
# Group the data by 'Date' and sum the 'Length', sort by sum in descending order, and show the top 2
data.group('Date',sum).sort('Length sum',descending = True).show(2)

Date,Time sum,Artist sum,Album sum,Track sum,Length sum,Popularity sum,Genres sum,Primary Genre sum
nan,,,,,8.04938e+08,127348,,
2-Sep-19,,,,,6.07972e+07,13891,,


In [ ]:
# Calculate the total length of tracks by 'Modern Baseball' with length > 0 in seconds
sum(data.where('Length',are.above(0)).where('Artist','Modern Baseball').column('Length'))/1000

531821.26300000004